In [1]:
cd ../src

/home/alberto/Work/incomplete_multiview_clustering/src


In [2]:
from mvlearn.datasets import load_nutrimouse
import pandas as pd
from utils import DatasetUtils

In [3]:
dict_data = load_nutrimouse()
gene = pd.DataFrame(dict_data['gene'], columns = dict_data['gene_feature_names'])
lipid = pd.DataFrame(dict_data['lipid'], columns = dict_data['lipid_feature_names'])
mvd = [gene, lipid]
sample_views = DatasetUtils.add_sample_views(mvd = mvd, p = [0.2, 0.5])
imvd = DatasetUtils.add_missing_views(mvd = mvd, sample_views = sample_views)
for data in imvd:
    print(data.shape)
    display(data.head())

(36, 120)


,X36b4,ACAT1,ACAT2,ACBP,ACC1,ACC2,ACOTH,ADISP,ADSS1,ALDH3,...,cHMGCoAS,cMOAT,eif2g,hABC1,iBABP,iBAT,iFABP,iNOS,mABC1,mHMGCoAS
0,-0.42,-0.65,-0.84,-0.34,-1.29,-1.13,-0.93,-0.98,-1.19,-0.68,...,-1.15,-0.78,-1.23,-1.16,-0.78,-1.65,-1.14,-1.24,-0.85,-0.03
2,-0.48,-0.74,-1.10,-0.46,-1.30,-1.09,-1.06,-1.08,-1.18,-0.75,...,-0.93,-0.89,-1.14,-1.25,-0.89,-1.89,-1.16,-1.35,-0.96,-0.12
3,-0.45,-0.69,-0.65,-0.41,-1.26,-1.09,-0.93,-1.02,-1.07,-0.71,...,-1.10,-0.93,-1.10,-1.17,-0.82,-1.70,-1.17,-1.25,-0.87,-0.12
4,-0.42,-0.71,-0.54,-0.38,-1.21,-0.89,-1.00,-0.95,-1.08,-0.76,...,-0.94,-0.84,-1.10,-1.14,-0.71,-1.68,-1.15,-1.20,-0.82,-0.29
5,-0.43,-0.69,-0.80,-0.32,-1.13,-0.79,-0.93,-0.97,-1.07,-0.75,...,-0.83,-0.81,-1.05,-1.08,-0.76,-1.68,-1.08,-1.16,-0.81,-0.15


(27, 21)


,C140,C160,C180,C161n9,C161n7,C181n9,C181n7,C201n9,C203n9,C182n6,...,C202n6,C203n6,C204n6,C224n6,C225n6,C183n3,C203n3,C205n3,C225n3,C226n3
1,0.38,24.04,9.93,0.55,2.54,20.07,3.92,0.23,0.00,14.98,...,0.30,1.64,15.34,0.58,2.10,0.00,0.00,0.00,0.48,2.61
3,0.22,25.48,8.14,0.49,2.82,21.92,2.52,0.00,0.00,13.89,...,0.00,1.10,3.92,0.00,0.00,0.49,0.00,2.99,1.04,14.99
4,0.37,24.80,9.63,0.46,2.85,21.38,2.96,0.30,0.27,14.55,...,0.23,1.58,11.85,0.32,0.44,0.42,0.00,0.30,0.35,6.69
5,1.70,26.04,6.59,0.66,7.26,28.23,8.99,0.36,2.89,3.47,...,0.00,0.81,5.09,0.00,0.56,0.00,0.00,0.00,2.13,2.56
6,0.35,25.94,9.68,0.36,3.60,17.62,2.15,0.25,0.00,8.73,...,0.00,0.68,2.57,0.00,0.00,8.40,0.42,7.37,2.05,9.84


In [31]:
from utils import DatasetUtils
from sumo.modes.prepare.similarity import feature_to_adjacency
from sumo.network import MultiplexNet
from sumo.modes.run.solvers.unsupervised_sumo import UnsupervisedSumoNMF
from sumo.modes.run.run import run_thread_wrapper
from scipy.cluster.hierarchy import cophenet, linkage
from scipy.spatial.distance import pdist
from sumo.utils import extract_ncut
import numpy as np


class SUMO():
    
    def __init__(self, k: int, method = ['euclidean'], missing : list = [0.1], neighbours : float = 0.1, alpha: float = 0.5, sparsity : list = [0.1], repetitions : int = 60,
                 cluster_method : str = "max_value", max_iter : int =500, tol : float = 1e-5, subsample : float = 0.05, calc_cost : int = 20,
                 h_init :int = None, n_jobs : int = 1, rep : int = 5, random_state : int = None, verbose : bool = False):
        self.method = method
        self.missing = missing
        self.neighbours = neighbours
        self.alpha = alpha
        self.k = k
        self.sparsity = sparsity
        self.repetitions = repetitions
        self.cluster_method = cluster_method
        self.max_iter = max_iter
        self.tol = tol
        self.subsample = subsample
        self.calc_cost = calc_cost
        self.h_init = h_init
        self.n_jobs = n_jobs
        self.rep = rep
        self.random_state = random_state
        self.verbose = verbose
        
        self.graph_ = None
        self.nmf_ = None
        self.priors_ = None
        
        if self.repetitions < 1:
            raise ValueError("Incorrect value of 'repetitions' parameter")
        if self.n_jobs < 1:
            raise ValueError("Incorrect number of threads")
        if self.subsample > 0.5 or self.subsample < 0:
            # do not allow for removal of more then 50% of samples in each run
            raise ValueError("Incorrect value of 'subsample' parameter")
        if self.rep < 1:
            # number of times additional consensus matrix will be created
            raise ValueError("Incorrect value of 'rep' parameter")
        if self.random_state is not None and self.random_state < 0:
            raise ValueError("Seed value cannot be negative")
        self.runs_per_con = max(round(self.repetitions * 0.8), 1)  # number of runs per consensus matrix creation
        
        if len(self.k) > 2 or (len(self.k) == 2 and self.k[0] > self.k[1]):
            raise ValueError("Incorrect range of k values")
        elif len(self.k) == 2:
            self.k = list(range(self.k[0], self.k[1] + 1))        
        
        
        
    def fit(self, X, y = None):
        if len(self.missing) == 1:
            if self.verbose:
                print(f"#Setting all 'missing' parameters to {self.missing[0]}")
            self.missing = [self.missing[0]] * len(X)
        if len(self.method) == 1:
            self.method = [self.method[0]] * len(X)
        elif len(X) != len(self.method):
            raise ValueError("Number of matrices extracted from input files and number of similarity methods does not correspond")

        all_samples = DatasetUtils.get_sample_views(imvd = X).index.values
        if self.verbose:
            print(f"Total number of unique samples: {len(all_samples)}")
        self.similarity_ = {}
        adj_matrices = []
        # create adjacency matrices
        for view_idx, view_data in enumerate(X):
            if self.verbose:
                print(f"#Layer: {view_idx}")
                print(f"Feature matrix: ({view_data.shape[0]} samples x {view_data.shape[1]} features)")
            # create adjacency matrix
            a = feature_to_adjacency(view_data.values, missing=self.missing[view_idx], method=self.method[view_idx], n=self.neighbours, alpha=self.alpha)
            if self.verbose:
                print(f"Adjacency matrix: ({a.shape} created [similarity method: {self.method[view_idx]}")
            # add matrices to output arrays
            adj_matrices.append(a)
            self.similarity_[str(view_idx)] = a
            self.similarity_[f"f{view_idx}"] = view_data
            
        ##################################################################
        if self.h_init is not None:
            if self.h_init >= len(adj_matrices) or self.h_init < 0:
                raise ValueError("Incorrect value of h_init")
                
        # create multilayer graph
        self.graph = MultiplexNet(adj_matrices=adj_matrices, node_labels= all_samples)
        n_sub_samples = round(all_samples.size * self.subsample)
        if self.verbose:
            print(f"#Number of samples randomly removed in each run: {n_sub_samples} out of {all_samples.size}")
        # create solver
        self.nmf = UnsupervisedSumoNMF(graph=self.graph, nbins=self.repetitions, bin_size=self.graph.nodes - n_sub_samples, rseed=self.random_state)
        global _sumo_run
        _sumo_run = self  # this solves multiprocessing issue with pickling
        # run factorization for every (eta, k)
        cophenet_list = []
        pac_list = []
        for k in self.k:
            if self.verbose:
                print(f"#K:{k}")
            if self.n_jobs == 1:
                results = [SUMO._run_factorization(sparsity=sparsity, k=k, sumo_run=_sumo_run, verbose = self.verbose) for sparsity in self.sparsity]
                sparsity_order = self.sparsity
            else:
                if self.verbose:
                    print(f"{self.sparsity} processes to run")
                pool = mp.Pool(self.t)
                results = []
                sparsity_order = []
                iproc = 1
                for res in pool.imap_unordered(run_thread_wrapper, zip(self.sparsity, [k] * len(self.sparsity))):
                    if self.verbose:
                        print(f"- process {iproc} finished")
                    results.append(res[0])
                    sparsity_order.append(res[1])
                    iproc += 1

            # select best result
            best_result = sorted(results, reverse=True)[0]
            best_eta = None

            quality_output = []
            for (result, sparsity) in zip(results, sparsity_order):
                if self.verbose:
                    print(f"#Clustering quality (eta={sparsity}): {result[0]}")
                quality_output.append(np.array([sparsity, result[0]]))
                if result[1] == best_result[1]:
                    best_eta = sparsity

            # summarize results
            assert best_eta is not None
            return best_result
#             out_arrays = load_npz(best_result[1])

#             cophenet_list.append(out_arrays["cophenet"])
#             pac_list.append(out_arrays["pac"])

#             # create text file with cluster labels
#             clusters = out_arrays['clusters']
#             with open(os.path.join(self.outdir, "k{}".format(k), "clusters.tsv"), 'w') as cl_file:
#                 cl_file.write("sample\tlabel\n")
#                 for row_idx in range(clusters.shape[0]):
#                     cl_file.write("{}\t{}\n".format(clusters[row_idx, 0], clusters[row_idx, 1]))

#             # create symlink to the selected best result
#             summary_outfile = os.path.join(self.outdir, "k{}".format(k), "sumo_results.npz")
#             if os.path.lexists(summary_outfile):
#                 # overwriting symlink
#                 os.remove(summary_outfile)

#             workdir = os.getcwd()
#             os.chdir(os.path.dirname(best_result[1]))
#             os.symlink(os.path.basename(best_result[1]), os.path.basename(summary_outfile))
#             os.chdir(workdir)
#             assert os.getcwd() == workdir

#             self.logger.info("Results (k = {}) saved to {}".format(k, summary_outfile))

#             plot_heatmap_seaborn(out_arrays['consensus'], labels=np.arange(self.graph.nodes),
#                                  title="Consensus matrix (K = {})".format(k),
#                                  file_path=os.path.join(self.plot_dir, "consensus_k{}.png".format(k)))
#             # TODO: change sample order

#         if len(cophenet_list) > 1 and len(pac_list) > 1:
#             cophenet_plot_path = os.path.join(self.plot_dir, "cophenet.png")
#             plot_metric(x=self.k, y=cophenet_list, xlabel="K", ylabel="cophenetic correlation coefficient",
#                         title="Cluster stability for different K values", file_path=cophenet_plot_path, color="red")
#             self.logger.info("#Cophentic correlation coefficient plot for different K values has " +
#                              "been saved to {}".format(cophenet_plot_path))

#             pac_plot_path = os.path.join(self.plot_dir, "pac.png")
#             plot_metric(x=self.k, y=pac_list, xlabel="K", ylabel="PAC",
#                         title="Proportion of ambiguous clusterings for different K values", file_path=pac_plot_path,
#                         color="blue")
#             self.logger.info("#Proportion of ambiguous clusterings plot for different K values has " +
#                              "been saved to {}".format(pac_plot_path))
#         return self


    @staticmethod
    def _run_factorization(sparsity: float, k: int, sumo_run, verbose : bool):
        """ Run factorization for set sparsity and number of clusters
        Args:
            sparsity (float): value of sparsity penalty
            k (int): number of clusters
            sumo_run: SumoRun object
        Returns:
            quality (float): assessed quality of cluster structure
            outfile (str): path to .npz output file with results of factorization
        """
        if sumo_run.random_state is not None:
            np.random.seed(sumo_run.random_state)

        # run factorization N times
        results = []
        for repeat in range(sumo_run.repetitions):
            if verbose:
                print(f"#Runing NMF algorithm with sparsity {sparsity} (N={repeat + 1})")
            opt_args = {
                "sparsity_penalty": sparsity,
                "k": k,
                "max_iter": sumo_run.max_iter,
                "tol": sumo_run.tol,
                "calc_cost": sumo_run.calc_cost,
                "bin_id": repeat,
                "h_init": sumo_run.h_init
            }
            result = sumo_run.nmf.factorize(**opt_args)
            # extract computed clusters
            if verbose:
                print(f"#Using {sumo_run.cluster_method} for cluster labels extraction)")
            result.extract_clusters(method=sumo_run.cluster_method)
            results.append(result)

        # consensus graph
        assert len(results) > 0

        all_REs = []  # residual errors
        for run_idx in range(sumo_run.repetitions):
            all_REs.append(results[run_idx].RE)

        out_arrays = {'pac': np.array([]), 'cophenet': np.array([])}
        minRE, maxRE = min(all_REs), max(all_REs)

        for rep in range(sumo_run.rep):
            run_indices = list(np.random.choice(range(len(results)), sumo_run.runs_per_con, replace=False))

            consensus = np.zeros((sumo_run.graph.nodes, sumo_run.graph.nodes))
            weights = np.empty((sumo_run.graph.nodes, sumo_run.graph.nodes))
            weights[:] = np.nan

            all_equal = np.allclose(minRE, maxRE)

            for run_idx in run_indices:
                weight = np.empty((sumo_run.graph.nodes, sumo_run.graph.nodes))
                weight[:] = np.nan
                sample_ids = results[run_idx].sample_ids
                if all_equal:
                    weight[sample_ids, sample_ids[:, None]] = 1.
                else:
                    weight[sample_ids, sample_ids[:, None]] = (maxRE - results[run_idx].RE) / (maxRE - minRE)

                weights = np.nansum(np.stack((weights, weight)), axis=0)
                consensus_run = np.nanprod(np.stack((results[run_idx].connectivity, weight)), axis=0)
                consensus = np.nansum(np.stack((consensus, consensus_run)), axis=0)

            if verbose:
                print(f"#Creating consensus graphs [{rep + 1} out of {sumo_run.rep}]")
            assert not np.any(np.isnan(consensus))
            consensus = consensus / weights

            org_con = consensus.copy()
            consensus[consensus < 0.5] = 0

            # calculate cophenetic correlation coefficient
            dist = pdist(org_con, metric="correlation")
            if np.any(np.isnan(dist)):
                ccc = np.nan
                if verbose:
                    print("Cannot calculate cophenetic correlation coefficient! Please inspect values in your consensus matrix")
            else:
                ccc = cophenet(linkage(dist, method="complete", metric="correlation"), dist)[0]

            # calculate proportion of ambiguous clustering
            den = (sumo_run.graph.nodes ** 2) - sumo_run.graph.nodes
            num = org_con[(org_con > 0.1) & (org_con < 0.9)].size
            pac = num * (1. / den)

            out_arrays.update({'pac': np.append(out_arrays['pac'], pac),
                               'cophenet': np.append(out_arrays['cophenet'], ccc)})

        if verbose:
            print("#Extracting final clustering result, using normalized cut")
        consensus_labels = extract_ncut(consensus, k=k)

        cluster_array = np.empty((sumo_run.graph.sample_names.shape[0], 2), dtype=np.object)
        # TODO add column with confidence value when investigating soft clustering
        cluster_array[:, 0] = sumo_run.graph.sample_names
        cluster_array[:, 1] = consensus_labels

        clusters_dict = {num: sumo_run.graph.sample_names[list(np.where(consensus_labels == num)[0])] for num in
                         np.unique(consensus_labels)}
        for cluster_idx in sorted(clusters_dict.keys()):
            if verbose:
                print(f"Cluster {cluster_idx} ({len(clusters_dict[cluster_idx])} samples): \n{clusters_dict[cluster_idx]}")

        # calculate quality of clustering for given sparsity
        quality = sumo_run.graph.get_clustering_quality(labels=cluster_array[:, 1])
        # create output file
        conf_array = np.empty((9, 2), dtype=object)
        conf_array[:, 0] = ['method', 'n', 'max_iter', 'tol', 'subsample', 'calc_cost', 'h_init', 'seed', 'sparsity']
        conf_array[:, 1] = [sumo_run.cluster_method, sumo_run.repetitions, sumo_run.max_iter, sumo_run.tol, sumo_run.subsample,
                            sumo_run.calc_cost, np.nan if sumo_run.h_init is None else sumo_run.h_init,
                            np.nan if sumo_run.random_state is None else sumo_run.random_state, sparsity]
        out_arrays.update({
            "clusters": cluster_array,
            "consensus": consensus,
            "unfiltered_consensus": org_con,
            "quality": np.array(quality),
            "samples": sumo_run.graph.sample_names,
            "config": conf_array
        })

        steps_reached = [results[i].steps for i in range(len(results))]
        maxiter_proc = round((sum([step == sumo_run.max_iter for step in steps_reached]) / len(steps_reached)) * 100, 3)
        if verbose:
            print(f"#Reached maximum number of iterations in {maxiter_proc}% of runs")
        if maxiter_proc >= 90:
            if verbose:
                print(f"Consider increasing -max_iter and decreasing -tol to achieve better accuracy")
        out_arrays['steps'] = np.array([steps_reached])
        return quality, out_arrays

In [33]:
sumo = SUMO(k = [3], verbose=False)
sumo.fit(X = mvd)

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
`np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


(0.02180216178039221,
 {'pac': array([0.32820513, 0.32948718, 0.31282051, 0.31282051, 0.31666667]),
  'cophenet': array([0.99651781, 0.9959307 , 0.9964618 , 0.99518086, 0.99666488]),
  'clusters': array([[0, 0],
         [1, 2],
         [2, 2],
         [3, 0],
         [4, 2],
         [5, 2],
         [6, 0],
         [7, 0],
         [8, 0],
         [9, 2],
         [10, 0],
         [11, 2],
         [12, 2],
         [13, 2],
         [14, 2],
         [15, 0],
         [16, 2],
         [17, 0],
         [18, 2],
         [19, 2],
         [20, 1],
         [21, 1],
         [22, 1],
         [23, 1],
         [24, 1],
         [25, 1],
         [26, 1],
         [27, 1],
         [28, 1],
         [29, 1],
         [30, 1],
         [31, 1],
         [32, 1],
         [33, 1],
         [34, 1],
         [35, 1],
         [36, 1],
         [37, 1],
         [38, 1],
         [39, 1]], dtype=object),
  'consensus': array([[1.        , 0.        , 0.        , 0.9962117 , 0.      

In [2]:
from sumo.modes.prepare.similarity import feature_to_adjacency

In [4]:
estimator = Concat()
estimator.fit(X = imvd)

Concat()

In [5]:
estimator.predict(X = imvd)

array([3, 1, 7, 1, 1, 6, 6, 7, 3, 7, 5, 3, 7, 1, 4, 6, 5, 7, 3, 6, 6, 6,
       3, 6, 6, 3, 3, 1, 7, 4, 3, 4, 0, 4, 7, 6, 3, 6, 7, 6, 0, 1, 3, 1,
       2, 2, 5, 5, 6, 3, 1, 6, 0, 0, 6, 3, 6, 3, 0, 7, 0, 3, 3, 6, 2, 6,
       1, 7, 3, 7, 6, 7, 6, 3, 4, 6, 5, 1, 1, 3, 3, 4, 3, 1, 0, 7, 7, 2,
       4, 6, 5, 7, 1, 3, 0, 4, 3, 0, 0, 4], dtype=int32)